In [ ]:
import json
import re
import pandas as pd

In [ ]:
def refactor(text):
    text = text.lower()
    notallowed = (':;\'1234567890-+_)—(*&^%$#}{[]|\\/.,?\«»"“”')
    for char in notallowed:
        text = text.replace(char, '')
    return text

In [ ]:
def get_words(texts):
    words_list = []
    for text in texts:
        cleaned_text = re.sub(r'\s+', ' ', text)
        cleaned_text = cleaned_text.strip()
        words = cleaned_text.split()
        words_list.extend(refactor(word) for word in words)
    return words_list


In [ ]:
text_dict = dict()

In [ ]:
from collections import defaultdict


def get_words_dict(chunk_id) :
    path = 'dataset/chunk_' + str(chunk_id) + '.json'
    words_dict = defaultdict(int)
    with open(path, 'r') as f:
        data = json.load(f)
        texts_list = []
        for text in data['rows']:
            item = text['row']['text']
            texts_list.append(item)
        words = get_words(texts_list)
        
        for word in words:
            if len(word) > 0:
                words_dict[word] += 1
            
    df = pd.DataFrame(list(words_dict.items()))
    df.to_csv('prepared2/chunk_' + str(chunk_id) + '.csv', index=False)
    return words_dict
        
        

In [ ]:
dict_list = []

In [ ]:
import os
import requests

In [ ]:
def remove_first_row_from_csv_files_in_directory(directory_path):
    # Get a list of all CSV files in the specified directory
    csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]
    print(len(csv_files))
    for file_name in csv_files:
        print(file_name)
        file_name = directory_path + '/' + file_name
        df = pd.read_csv(file_name)
        print(df.head(10))
        df = df.iloc[1:]
        print(df.head(10))
        # df.to_csv(file_name, index=False)
        break

directory_path = 'prepared2/'
remove_first_row_from_csv_files_in_directory(directory_path)


In [ ]:
for i in range(6130, 6131) :  
    print('chunk ' + str(i))
    try:
        dict_list.append(get_words_dict(i))
    except Exception as e :
        print('error', i, e)
        # os.remove('prepared/chunk_' + str(i) + '.csv')
        if os.path.exists('dataset/chunk_' + str(i) + '.json') :
            os.remove('dataset/chunk_' + str(i) + '.json')
        offset = i * 100
        data = requests.get(f'https://datasets-server.huggingface.co/rows?dataset=tahrirchi%2Fuz-crawl&config=default&split=news&offset={offset}&length=100')
    
        # with open('tahrirchi-news.json', 'a') as f :
        #     f.write(data.text)
        with open('dataset/chunk_'+str(i)+'.json', 'a') as f :
            f.write(data.text)
        print(i, data.status_code)
        
        
    